In [17]:
from config import RAW_DATA_PATH
import pandas as pd

import os

Load the dataframe and tokenized documents

In [44]:
df = pd.read_csv('data/preprocessing/document_tokens_labelled.csv', sep=',')

In [45]:
df

,document_tokens,label
0,commission|accès|document|administratif|examin...,défavorable
1,commission|accès|document|administratif|examin...,défavorable
2,commission|accès|document|administratif|examin...,défavorable
3,commission|accès|document|administratif|examin...,NaN
4,commission|accès|document|administratif|examin...,NaN
...,...,...
48741,monsieur|x|saisir|commission|accès|document|ad...,favorable
48742,monsieur|x|saisir|commission|accès|document|ad...,sans objet
48743,maître|x|x|saisir|commission|accès|document|ad...,sans objet
48744,Monsieur|x|x|saisir|commission|accès|document|...,sans objet


In [46]:
df.label.value_counts()

favorable      26940
sans objet      9849
défavorable     3665
Name: label, dtype: int64

Split into Train and Test upfront in order to avoid data leakage.

Build a naive baseline model

Build a training pipeline with sklearn

Build the inference pipeline

Run inference pipeline on test data and evaluate results